### Prepare Data

Plan - Acquire - Prepare - Explore - Model - Deliver

What we are doing and why:
What: Clean and tidy our data so that it is ready for exploration, analysis and modeling

Why: Set ourselves up for certainty!

1) Ensure that our observations will be sound:
    Validity of statistical and human observations
2) Ensure that we will not have computational errors:
    non numerical data cells, nulls/NaNs
3) Protect against overfitting:
    Ensure that have a split data structure prior to drawing conclusions

### High level Roadmap:

Input: An aquired dataset (One Pandas Dataframe) ------> Output: Tidied and cleaned data split into Train,  Validate, and Test sets (Three Pandas Dataframes)
Processes: Summarize the data ---> Clean the data ---> Split the data¶

### Summarize

In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings('ignore')

import acquire as aq

In [2]:
# Grab our acquired dataset:

In [3]:
df = aq.get_titanic_data()

In [4]:
# take a look at our data:

In [5]:
df.shape

(891, 13)

In [6]:
df.head()

,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,3,male,22.0,1,0,7.2500,S,Third,None,Southampton,0
1,1,1,1,female,38.0,1,0,71.2833,C,First,C,Cherbourg,0
2,2,1,3,female,26.0,0,0,7.9250,S,Third,None,Southampton,1
3,3,1,1,female,35.0,1,0,53.1000,S,First,C,Southampton,0
4,4,0,3,male,35.0,0,0,8.0500,S,Third,None,Southampton,1


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   passenger_id  891 non-null    int64  
 1   survived      891 non-null    int64  
 2   pclass        891 non-null    int64  
 3   sex           891 non-null    object 
 4   age           714 non-null    float64
 5   sibsp         891 non-null    int64  
 6   parch         891 non-null    int64  
 7   fare          891 non-null    float64
 8   embarked      889 non-null    object 
 9   class         891 non-null    object 
 10  deck          203 non-null    object 
 11  embark_town   889 non-null    object 
 12  alone         891 non-null    int64  
dtypes: float64(2), int64(6), object(5)
memory usage: 90.6+ KB


In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
passenger_id,891.0,445.000000,257.353842,0.00,222.5000,445.0000,667.5,890.0000
survived,891.0,0.383838,0.486592,0.00,0.0000,0.0000,1.0,1.0000
pclass,891.0,2.308642,0.836071,1.00,2.0000,3.0000,3.0,3.0000
age,714.0,29.699118,14.526497,0.42,20.1250,28.0000,38.0,80.0000
sibsp,891.0,0.523008,1.102743,0.00,0.0000,0.0000,1.0,8.0000
parch,891.0,0.381594,0.806057,0.00,0.0000,0.0000,0.0,6.0000
fare,891.0,32.204208,49.693429,0.00,7.9104,14.4542,31.0,512.3292
alone,891.0,0.602694,0.489615,0.00,0.0000,1.0000,1.0,1.0000


In [9]:
num_cols = df.select_dtypes(np.number).columns
num_cols

Index(['passenger_id', 'survived', 'pclass', 'age', 'sibsp', 'parch', 'fare',
       'alone'],
      dtype='object')

In [10]:
obj_cols = df.select_dtypes(np.object).columns
obj_cols

Index(['sex', 'embarked', 'class', 'deck', 'embark_town'], dtype='object')

In [11]:
# Describe our object columns:


In [12]:
# describe object columns
for col in obj_cols:
    print(df[col].value_counts(), '\n\n~~~~~\n')
    print(df[col].value_counts(normalize = True, dropna = False), '\n\n- - -\n')

male      577
female    314
Name: sex, dtype: int64 

~~~~~

male      0.647587
female    0.352413
Name: sex, dtype: float64 

- - -

S    644
C    168
Q     77
Name: embarked, dtype: int64 

~~~~~

S      0.722783
C      0.188552
Q      0.086420
NaN    0.002245
Name: embarked, dtype: float64 

- - -

Third     491
First     216
Second    184
Name: class, dtype: int64 

~~~~~

Third     0.551066
First     0.242424
Second    0.206510
Name: class, dtype: float64 

- - -

C    59
B    47
D    33
E    32
A    15
F    13
G     4
Name: deck, dtype: int64 

~~~~~

NaN    0.772166
C      0.066218
B      0.052750
D      0.037037
E      0.035915
A      0.016835
F      0.014590
G      0.004489
Name: deck, dtype: float64 

- - -

Southampton    644
Cherbourg      168
Queenstown      77
Name: embark_town, dtype: int64 

~~~~~

Southampton    0.722783
Cherbourg      0.188552
Queenstown     0.086420
NaN            0.002245
Name: embark_town, dtype: float64 

- - -



In [13]:
# missing values
missing = df.isnull().sum()
missing[missing > 0]

age            177
embarked         2
deck           688
embark_town      2
dtype: int64

##### Gather our takeaways, i.e., what we are going to do when we clean:

1. embarked == embark_town. Pick one to keep since they are dupicates. We'll keep embarked_town for now since it is more human readable.

2. class == pclass. They say the same thing, so let's keep the one that is numerical.

3. deck and age have many missing values. Deck will be of no use to us with that many missing values and we will say the same of age without more insight. We will drop these columns.

4. We have just two values missing from embark_town, and we will fill these in with the most common embark_town category

5. For embark_town and sex, we will encode the values

### Clean

In [14]:
# drop duplicates...run just in case
df.drop_duplicates(inplace=True)

In [15]:
# drop columns with too many missing to have any value right now
df.drop(columns=['deck', 'age', 'embarked', 'class'], inplace=True)

We could fill embark_town with most common value, 'Southampton', by hard-coding the value using the fillna() function, as below. Or we could use an imputer. We will demonstrate the imputer after the train-validate-test split.

In [16]:
df.embark_town.fillna(value = 'Southampton', inplace=True)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   passenger_id  891 non-null    int64  
 1   survived      891 non-null    int64  
 2   pclass        891 non-null    int64  
 3   sex           891 non-null    object 
 4   sibsp         891 non-null    int64  
 5   parch         891 non-null    int64  
 6   fare          891 non-null    float64
 7   embark_town   891 non-null    object 
 8   alone         891 non-null    int64  
dtypes: float64(1), int64(6), object(2)
memory usage: 69.6+ KB


In [18]:
dummy_df = pd.get_dummies(df[['sex', 'embark_town']], dummy_na=False, drop_first=[True, True])

In [19]:
df = pd.concat([df, dummy_df], axis = 1)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   passenger_id             891 non-null    int64  
 1   survived                 891 non-null    int64  
 2   pclass                   891 non-null    int64  
 3   sex                      891 non-null    object 
 4   sibsp                    891 non-null    int64  
 5   parch                    891 non-null    int64  
 6   fare                     891 non-null    float64
 7   embark_town              891 non-null    object 
 8   alone                    891 non-null    int64  
 9   sex_male                 891 non-null    uint8  
 10  embark_town_Queenstown   891 non-null    uint8  
 11  embark_town_Southampton  891 non-null    uint8  
dtypes: float64(1), int64(6), object(2), uint8(3)
memory usage: 72.2+ KB


In [21]:
# Let's make that into a function so we can repeat it all easily in one step.

In [22]:
df = aq.get_titanic_data(force_cache=True)

TypeError: get_titanic_data() got an unexpected keyword argument 'force_cache'

In [ ]:
def clean_titanic_data(df):
    df = df.drop_duplicates()
    df = df.drop(columns=['deck', 'age', 'embarked', 'class'])
    df['embark_town'] = df.embark_town.fillna(value='Southampton')
    dummy_df = pd.get_dummies(df[['sex', 'embark_town']], drop_first=True)
    df = pd.concat([df, dummy_df], axis=1)
    return df.drop(columns=['sex', 'embark_town'])

In [ ]:
cleaned_df_v0 = clean_titanic_data(df)

In [ ]:
cleaned_df_v0.info()

Create a function to perform these steps when we need to reproduce our dataset.

### Train, Validate, Test Split

In [23]:
# 20% test, 80% train_validate
# then of the 80% train_validate: 30% validate, 70% train. 
train, test = train_test_split(cleaned_df_v0, test_size=0.2, random_state=19, stratify=cleaned_df_v0.survived)
train, validate = train_test_split(train, test_size=0.3, random_state=19, stratify=train.survived)

NameError: name 'cleaned_df_v0' is not defined

### Option for Missing Values: Impute

We can impute values using the mean, median, mode (most frequent), or a constant value. We will use sklearn.imputer.SimpleImputer to do this.

1. Create the imputer object, selecting the strategy used to impute (mean, median or mode (strategy = 'most_frequent').

2. Fit to train. This means compute the mean, median, or most_frequent (i.e. mode) for each of the columns that will be imputed. Store that value in the imputer object.

3. Transform train: fill missing values in train dataset with that value identified

4. Transform test: fill missing values with that value identified

##### 1. Create the SimpleImputer object, which we will store in the variable imputer. In the creation of the object, we will specify the strategy to use (mean, median, most_frequent). Essentially, this is creating the instructions and assigning them to a variable we will reference.

1. Fit the imputer to the columns in the training df. This means that the imputer will determine the most_frequent value, or other value depending on the strategy called, for each column.

In [24]:
imputer = imputer.fit(df[['embark_town']])
imputer

NameError: name 'imputer' is not defined

SimpleImputer(missing_values=None, strategy='most_frequent')

It will store that value in the imputer object to use upon calling transform. We will call transform on each of our samples to fill any missing values.

In [25]:

train[['embark_town']] = imputer.transform(train[['embark_town']])
validate[['embark_town']] = imputer.transform(validate[['embark_town']])
test[['embark_town']] = imputer.transform(test[['embark_town']])


NameError: name 'imputer' is not defined

Create a function that will run through all of these steps, when I provide a train and test dataframe, a strategy, and a list of columns.

In [26]:

def impute_mode(train, validate, test):
    imputer = SimpleImputer(missing_values=None, strategy='most_frequent')
    train[['embark_town']] = imputer.fit_transform(train[['embark_town']])
    validate[['embark_town']] = imputer.transform(validate[['embark_town']])
    test[['embark_town']] = imputer.transform(test[['embark_town']])
    return train, validate, test

Blend the clean, split and impute functions into a single prep_data() function.

In [27]:
def prep_titanic_data(df):
    df = df.drop_duplicates()
    df = df.drop(columns=['passenger_id', 'deck', 'age', 'embarked', 'class'])
    
    train, test = train_test_split(df, test_size=0.2, random_state=19, stratify=df.survived)
    train, validate = train_test_split(train, test_size=0.3, random_state=19, stratify=train.survived)
    train, validate, test = impute_mode(train, validate, test)
    
    dummy_train = pd.get_dummies(train[['sex', 'embark_town']], drop_first=True)
    dummy_validate = pd.get_dummies(validate[['sex', 'embark_town']], drop_first=True)
    dummy_test = pd.get_dummies(test[['sex', 'embark_town']], drop_first=True)
    
    train = pd.concat([train, dummy_train], axis=1)
    validate = pd.concat([validate, dummy_validate], axis=1)
    test = pd.concat([test, dummy_test], axis=1)
    
    train = train.drop(columns=['sex', 'embark_town'])
    validate = validate.drop(columns=['sex', 'embark_town'])
    test = test.drop(columns=['sex', 'embark_town'])
    
    return train, validate, test

In [28]:
df = aq.get_titanic_data()

In [29]:
train, validate, test = prep_titanic_data(df)

In [30]:
test.describe().T

,count,mean,std,min,25%,50%,75%,max
survived,179.0,0.385475,0.488073,0.0,0.0000,0.0,1.0,1.0
pclass,179.0,2.312849,0.849588,1.0,1.5000,3.0,3.0,3.0
sibsp,179.0,0.469274,1.118237,0.0,0.0000,0.0,1.0,8.0
parch,179.0,0.296089,0.761825,0.0,0.0000,0.0,0.0,5.0
fare,179.0,30.212358,44.698635,0.0,7.8958,13.0,30.0,263.0
alone,179.0,0.692737,0.462653,0.0,0.0000,1.0,1.0,1.0
sex_male,179.0,0.664804,0.473383,0.0,0.0000,1.0,1.0,1.0
embark_town_Queenstown,179.0,0.083799,0.277863,0.0,0.0000,0.0,0.0,1.0
embark_town_Southampton,179.0,0.720670,0.449928,0.0,0.0000,1.0,1.0,1.0
